In [ ]:
#importing the required libraries
%pylab inline
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_percentage_error
from statsmodels.tools.eval_measures import rmse
import statsmodels.api as sm  #acf and pacf libraries
from scipy.optimize import linprog
from statsmodels.tsa.stattools import adfuller
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
!pip install --upgrade kneed
from kneed import KneeLocator #finding knee point
%matplotlib inline

In [ ]:
#reading the data given by the supermarket
energy= pd.read_csv('A0003 Pepper Hill Data.csv')
Temperature= pd.read_csv('Temperature Data.csv')

In [ ]:
#imputing missing values in temperature: average of previous and next available data
Temperature["Temperature"] = Temperature['Temperature'].astype(float)
Temperature['Temperature'] = Temperature['Temperature'].replace({np.nan: 'zero'})

for i in range (1, 17518):
    if Temperature['Temperature'][i]== 'zero':
        if Temperature['Temperature'][i-1]!= 'zero' and Temperature['Temperature'][i+1]!= 'zero':
            Temperature['Temperature'][i]=(Temperature['Temperature'][i-1]+Temperature['Temperature'][i+1])/2

for i in range (1, 17518):
    if Temperature['Temperature'][i]== 'zero':
        Temperature['Temperature'][i]=Temperature['Temperature'][i-1]

In [ ]:
#preparing the main data frame named energy
energy['Temperature']=Temperature['Temperature']
energy['Temperature'][0]= 9.0
energy['Temperature'][17518]= 8.0
energy['Temperature']=energy['Temperature'].astype(float)
energy

In [ ]:
#define the dataframe as a date time style
energy.Date = pd.to_datetime(energy.Date)
energy['Week'] = energy.Date.dt.week
energy['Year'] = energy.Date.dt.year
energy['Month'] = energy.Date.dt.month
energy['Day'] = energy.Date.dt.day
energy['Hour'] =energy.Date.dt.hour
energy['Min']=energy.Date.dt.minute
energy.index= energy['Date']
del energy['Date']
energy

In [ ]:
#plotting the given data
fig, ax = plt.subplots(figsize=(21,6))
plt.plot(energy['Energy'], color= 'black')
#plt.title('Energy consumption within a year (half hourly recorded)', fontsize=20)
plt.ylabel('$X_t$', fontsize=15)
plt.xlabel('Date', fontsize=15)
plt.show()

In [ ]:
#plotting the differentiated data for one of the time slots
fig, ax = plt.subplots(figsize=(21,6))
plt.plot(energy['Energy'].loc[(energy['Hour'] == 2) & (energy['Min'] == 30)].diff(1).dropna(), color= 'black')
#plt.title('First order differencing for Energy consumption at 02:30', fontsize=20)
plt.ylabel(r'$X_t - X_{t-1}$', fontsize=15)
plt.xlabel('Date', fontsize=15)
plt.show()

In [ ]:
#plotting the temperature data
fig, ax = plt.subplots(figsize=(21,6))
plt.plot(energy['Temperature'], color= 'purple')
#plt.title('Temperature data within a year', fontsize=20)
plt.ylabel('$Temperature {^\circ}C$', fontsize=15)
plt.xlabel('Date', fontsize=15)
plt.show()

In [ ]:
#both temperature and energy for 02:30 in one plot to see the correlation
fig, ax = plt.subplots(figsize=(21,6))
plt.plot(energy['Temperature'].loc[(energy['Hour'] == 2) & (energy['Min'] == 30)], color= 'purple', label='$T_t$')
plt.plot(energy['Energy'].loc[(energy['Hour'] == 2) & (energy['Min'] == 30)], color= 'black', label='$X_t$')
#plt.title('Two varibales in one plot (for 02:30)', fontsize=20)

plt.xlabel('Date', fontsize=15)
plt.legend()
plt.show()

In [ ]:
# energy for 02:30
fig, ax = plt.subplots(figsize=(21,6))

plt.plot(energy['Energy'].loc[(energy['Hour'] == 2) & (energy['Min'] == 30)], color= 'black')
#plt.title('Two varibales in one plot (for 02:30)', fontsize=20)
plt.xlabel('Date', fontsize=15)
plt.ylabel('$X_t$', fontsize=15)
#plt.legend()
plt.show()

In [ ]:
#correlation among given variables
energy.iloc[:,0:2].corr()

In [ ]:
#plotting the data for only two days to magnify the seasonality
fig, ax = plt.subplots(figsize=(21,6))
plt.plot(energy.iloc[0:144,6].index,energy.iloc[0:144,0], color= 'black')
#plt.title('Energy consumption within 3 DAYS (half hourly recorded)', fontsize=20)
plt.xlabel('Date', fontsize=15)
plt.ylabel('$X_t$', fontsize=15)
plt.show()

In [ ]:
energy.info()

# Machine learning part: ARIMAX MODEL for each 48 time slot

In [ ]:
#Time: 00:00 (because for this time we have less data for training, it is out of the loop and is done separately)
df2=energy.loc[(energy['Hour'] == 0) & (energy['Min'] == 0)]
df2=df2.iloc[:,:2]
df2

In [ ]:
#plotting pacf
sm.tsa.graphics.plot_pacf(energy['Energy'].loc[(energy['Hour'] == 0) & (energy['Min'] == 0)].diff(1).dropna(), lags=50)
plt.show()

In [ ]:
#for 00:00
AIC_orders=[]

# Loop over p values from 0-3
for q in range(4):
  # Loop over q values from 0-6
    for p in range(7):
      	# create and fit ARIMA(p,q) model (d=1 because difference of data is stationary (adfuller test)); traning data is
        #given to check the aic
        model = SARIMAX(df2.iloc[0:357,0],exog=df2.iloc[0:357,1], order=(p,1,q),enforce_invertibility=False, enforce_stationarity=False)
        results = model.fit()

        # Append order and results tuple
        AIC_orders.append((p,q,results.aic))

# DataFrame from AIC_orders
order_df = pd.DataFrame(AIC_orders,
                        columns=['p','q','AIC'])

order_df=order_df.sort_values('AIC')
order_df = order_df.reset_index()
del order_df['index']

kn = KneeLocator(range(0,28),order_df['AIC'], curve='convex', direction='increasing') #finding the knee point(elbow)
h=kn.knee


data = [{'time':'0','p':order_df['p'][h], 'q':order_df['q'][h]}] #take knee point as the order of the time series

df_p_q = pd.DataFrame(data, columns=['time','p','q']) #recording selected orders


#defining the model based on the optimum p and q correspond to knee point
model= SARIMAX(df2.iloc[0:357,0],exog=df2.iloc[0:357,1],order=(order_df['p'][h],1,order_df['q'][h]), enforce_invertibility=False, enforce_stationarity=False)
results= model.fit()

predictions= results.predict(start =357, end=363, exog= df2.iloc[357:,1]) # static predict for a week
predictions.index= df2.iloc[357:,0].index #giving index for predictions
df=pd.DataFrame(data=predictions) #make a data frame from predictions

Error_data = [{'time':'0','RMSE':rmse(predictions,df2.iloc[357:,0]), 'Normalized RMSE':rmse(predictions,df2.iloc[357:,0])/average(df2.iloc[357:,0])}]
df_error= pd.DataFrame(Error_data, columns=['time','RMSE','Normalized RMSE'])   #collecting rmse for this period
print('00:00-static')
results.plot_diagnostics(figsize=(21,12)) #plotting qqplot, distribution, acf for residuals
plt.show()
print(results.summary()) #prints the summary of the model

# Rolling forecast

train, test = df2.iloc[0:357,0].values, df2.iloc[357:,0].values #defining test and train data
train_exo, test_exo= df2.iloc[0:357,1].values, df2.iloc[357:,1].values #defining test and train data for exogenous factor
history = [x for x in train] #make a history list
history_exo = [y for y in train_exo] #history for exogenous factor
predictions=list()
#loop below updates the variables for rolling forecast
for t in range(len(test)):
    model = SARIMAX(history,exog= history_exo,order=(order_df['p'][h],1,order_df['q'][h]),enforce_invertibility=False, enforce_stationarity=False)
    fit_model = model.fit()
    out= fit_model.forecast(exog= test_exo[t])
    yhat = out[0]
    predictions.append(yhat)
    J = test[t]
    s= test_exo[t]
    history.append(J)
    history_exo.append(s)
#make a dataframe from predictions
df_rolling=pd.DataFrame(data= predictions, index= df2.iloc[357:,0].index)  #adding predicted values to previous predictions
#recording rmse
Error_Rolling_data = [{'time':'0','RMSE':rmse(predictions,df2.iloc[357:,0]), 'Normalized RMSE':rmse(predictions,df2.iloc[357:,0])/average(df2.iloc[357:,0])}]
df_Rolling_error= pd.DataFrame(Error_data, columns=['time','RMSE','Normalized RMSE'])
print('00:00-rolling')
fit_model.plot_diagnostics(figsize=(21,12)) #plotting qqplot, distribution, acf for residuals
plt.show()
print(fit_model.summary())  #prints the summary of the model


In [ ]:
print ('The index for the knee point:' ,h)

In [ ]:
#plot the sorted AIC to get the knee point
fig, ax = plt.subplots(figsize=(21,6))
plt.plot(order_df['AIC'], linewidth=3.5, color= 'blue')
plt.title('Sorted AIC for 02:30', fontsize=20)
plt.xticks(range(0,28))
plt.ylabel('AIC')
plt.show()

In [ ]:
#shows that higher orders gain better AIC
order_df

In [ ]:
List1=[]
for i in range(0,28):
  kio=(order_df['p'][i],order_df['q'][i])
  List1.append(kio)

In [ ]:
List2=[]
for i in range(28):
  mio=str(List1[i])
  List2.append(mio)

In [ ]:
#plot the sorted AIC to get the knee point
fig, ax = plt.subplots(figsize=(21,6))
plt.plot(h, order_df['AIC'][h], marker="o", markersize=40, markeredgecolor="red",markerfacecolor="white", alpha=0.9)
ax.annotate('Knee point', xy=(h, order_df['AIC'][h]), xytext=(h-5, order_df['AIC'][h+2]),arrowprops=dict(facecolor='black', shrink=0.05),fontsize=15)
plt.plot(order_df['AIC'], linewidth=3.5, color= 'blue')
#plt.title('Sorted AIC for 02:30', fontsize=20)
# Set number of ticks for x-axis
ax.set_xticks(range(0,28))
# Set ticks labels for x-axis
ax.set_xticklabels(List2, rotation='vertical', fontsize=12)
plt.ylabel('AIC',fontsize=15)
plt.xlabel('$(p,q)$', fontsize=15)
plt.show()


In [ ]:
List1=[]
for i in range(0,28):
  kio=(order_df['p'][i],order_df['q'][i])
  List1.append(kio)
List2=[]
for i in range(28):
  mio=str(List1[i])
  List2.append(mio)
#plot the sorted AIC to get the knee point
fig, ax = plt.subplots(figsize=(21,6))
plt.plot(h, order_df['AIC'][h], marker="o", markersize=40, markeredgecolor="red",markerfacecolor="white", alpha=0.9)
ax.annotate('Knee point', xy=(h, order_df['AIC'][h]), xytext=(h-5, order_df['AIC'][h+2]),arrowprops=dict(facecolor='black', shrink=0.05),fontsize=15)
plt.plot(order_df['AIC'], linewidth=3.5, color= 'blue')
#plt.title('Sorted AIC for 02:30', fontsize=20)
# Set number of ticks for x-axis
ax.set_xticks(range(0,28))
# Set ticks labels for x-axis
ax.set_xticklabels(List2, rotation='vertical', fontsize=12)
plt.ylabel('AIC',fontsize=15)
plt.xlabel('$(p,q)$', fontsize=15)
plt.show()

In [ ]:
#for 00:30
df2=energy.loc[(energy['Hour'] == 0) & (energy['Min'] == 30)]
df2=df2.iloc[:,:2]
sm.tsa.graphics.plot_acf(df2.iloc[:,0], lags=30)
plt.show()

AIC_orders=[]

    # Loop over p values from 0-3
for q in range(4):
      # Loop over q values from 0-6
    for p in range(7):
            # create and fit ARIMA(p,q) model (d=1 because difference of data is stationary (adfuller test)); traning data is
            #given to check the aic
        model = SARIMAX(df2.iloc[0:358,0],exog=df2.iloc[0:358,1],order=(p,1,q),enforce_invertibility=False, enforce_stationarity=False)
        results = model.fit()

            # Append order and results tuple
        AIC_orders.append((p,q,results.aic))

    # DataFrame from AIC_orders
order_df = pd.DataFrame(AIC_orders,
                            columns=['p','q','AIC'])
    #sorting based on the minimum values of AIC
order_df=order_df.sort_values('AIC')
order_df = order_df.reset_index()
del order_df['index']

kn = KneeLocator(range(0,28),order_df['AIC'], curve='convex', direction='increasing') #finding the elbow
h=kn.knee

data = ['00:30',order_df['p'][h], order_df['q'][h]]
df_p_q.loc[len(df_p_q)]=data

    #defining the model based on the optimum p and q that AIC returned.
model= SARIMAX(df2.iloc[0:358,0],exog=df2.iloc[0:358,1], order=(order_df['p'][h],1,order_df['q'][h]), enforce_invertibility=False, enforce_stationarity=False)
results= model.fit()
predictions= results.predict(start =358, end=364,exog= df2.iloc[358:,1])
predictions.index= df2.iloc[358:,0].index
df=df.append(pd.DataFrame(data=predictions))


Error_data=['00:30',rmse(predictions,df2.iloc[358:,0]), rmse(predictions,df2.iloc[358:,0])/average(df2.iloc[358:,0])]
df_error.loc[len(df_error)]=Error_data
print('00:30-static')
results.plot_diagnostics(figsize=(21,12))
plt.show()
print(results.summary())

    # Rolling forecast

train, test = df2.iloc[0:358,0].values, df2.iloc[358:,0].values
train_exo, test_exo= df2.iloc[0:358,1].values, df2.iloc[358:,1].values
history = [x for x in train]
history_exo = [y for y in train_exo]
predictions=list()
for t in range(len(test)):
    model = SARIMAX(history,exog= history_exo,order=(order_df['p'][h],1,order_df['q'][h]),enforce_invertibility=False, enforce_stationarity=False)
    fit_model = model.fit()
    out= fit_model.forecast(exog= test_exo[t])
    yhat = out[0]
    predictions.append(yhat)
    J = test[t]
    s= test_exo[t]
    history.append(J)
    history_exo.append(s)
df_rolling= df_rolling.append(pd.DataFrame(data= predictions, index= df2.iloc[358:,0].index))   #adding new predictions to previous hours values

Error_Rolling_data=['00:30',rmse(predictions,df2.iloc[358,0]), rmse(predictions,df2.iloc[358,0])/average(df2.iloc[358,0])]
df_Rolling_error.loc[len(df_Rolling_error)]=Error_Rolling_data
print('00:30-rolling')
fit_model.plot_diagnostics(figsize=(21,12))
plt.show()
print(fit_model.summary())

In [ ]:
#for 00:30
List1=[]
for i in range(0,28):
  kio=(order_df['p'][i],order_df['q'][i])
  List1.append(kio)
List2=[]
for i in range(28):
  mio=str(List1[i])
  List2.append(mio)
#plot the sorted AIC to get the knee point
fig, ax = plt.subplots(figsize=(21,6))
plt.plot(h, order_df['AIC'][h], marker="o", markersize=20, markeredgecolor="red",markerfacecolor="white", alpha=0.9)
ax.annotate('Knee point', xy=(h, order_df['AIC'][h]), xytext=(h-5, order_df['AIC'][h+2]),arrowprops=dict(facecolor='black', shrink=0.05),fontsize=15)
plt.plot(order_df['AIC'], linewidth=3.5, color= 'blue')
#plt.title('Sorted AIC for 02:30', fontsize=20)
# Set number of ticks for x-axis
ax.set_xticks(range(0,28))
# Set ticks labels for x-axis
ax.set_xticklabels(List2, rotation='vertical', fontsize=12)
plt.ylabel('AIC',fontsize=15)
plt.xlabel('$(p,q)$', fontsize=15)
plt.show()

In [ ]:
#run the model for other 23 hours of the day
for i in range(1,24):
    for j in range(0,31,30):
        df2=energy.loc[(energy['Hour'] == i) & (energy['Min'] == j) ]
        df2=df2.iloc[:,:2]
        AIC_orders=[]

        # Loop over p values from 0-3
        for q in range(4):
          # Loop over q values from 0-6
            for p in range(7):
                model = SARIMAX(df2.iloc[0:358,0],exog=df2.iloc[0:358,1],order=(p,1,q),enforce_invertibility=False, enforce_stationarity=False)
                results = model.fit()

                # Append order and results tuple
                AIC_orders.append((p,q,results.aic))

        # DataFrame from AIC_orders
        order_df = pd.DataFrame(AIC_orders,
                                columns=['p','q','AIC'])
        #sorting based on the minimum values of AIC
        order_df=order_df.sort_values('AIC')
        order_df = order_df.reset_index()
        del order_df['index']
        kn = KneeLocator(range(0,28),order_df['AIC'], curve='convex', direction='increasing') #finding the elbow
        h=kn.knee

        data = [str(i)+':'+str(j),order_df['p'][h], order_df['q'][h]]
        df_p_q.loc[len(df_p_q)]=data


        #defining the model based on the optimum p and q that AIC returned.
        model= SARIMAX(df2.iloc[0:358,0],exog=df2.iloc[0:358,1], order=(order_df['p'][h],1,order_df['q'][h]), enforce_invertibility=False, enforce_stationarity=False)
        results= model.fit()
        predictions= results.predict(start =358, end=364,exog= df2.iloc[358:,1])
        predictions.index= df2.iloc[358:,0].index
        df=df.append(pd.DataFrame(data=predictions))

        Error_data=[str(i)+':'+str(j),rmse(predictions,df2.iloc[358:,0]), rmse(predictions,df2.iloc[358:,0])/average(df2.iloc[358:,0])]
        df_error.loc[len(df_error)]=Error_data
        print(str(i)+':'+str(j)+'static')
        sm.tsa.graphics.plot_acf(df2.iloc[:,0], lags=30)
        plt.show()
        results.plot_diagnostics(figsize=(21,12))
        plt.show()
        print(results.summary())
        # Rolling forecast
        #split into train and test sets

        train, test = df2.iloc[0:358,0].values, df2.iloc[358:,0].values
        train_exo, test_exo= df2.iloc[0:358,1].values, df2.iloc[358:,1].values
        history = [x for x in train]
        history_exo = [y for y in train_exo]
        predictions=list()
        for t in range(len(test)):
            model = SARIMAX(history,exog= history_exo,order=(order_df['p'][h],1,order_df['q'][h]),enforce_invertibility=False, enforce_stationarity=False)
            fit_model = model.fit()
            out= fit_model.forecast(exog= test_exo[t])
            yhat = out[0]
            predictions.append(yhat)
            J = test[t]
            s= test_exo[t]
            history.append(J)
            history_exo.append(s)
        df_rolling= df_rolling.append(pd.DataFrame(data= predictions, index=df2.iloc[358:,0].index))  #adding new predictions to previous hours values

        Error_Rolling_data=[str(i)+':'+str(j),rmse(predictions,df2.iloc[358:,0]), rmse(predictions,df2.iloc[358:,0])/average(df2.iloc[358:,0])]
        df_Rolling_error.loc[len(df_Rolling_error)]=Error_Rolling_data
        print(str(i)+':'+str(j)+'rolling')
        fit_model.plot_diagnostics(figsize=(21,12))
        plt.show()
        print(fit_model.summary())





In [ ]:
df

In [ ]:
df_rolling

In [ ]:
df_error

In [ ]:
#plot rmse for each time slot
fig, ax = plt.subplots(figsize=(21,5))
plt.plot(df_Rolling_error.iloc[:,0],df_Rolling_error.iloc[:,2], color='green', label='Normalised RMSE, Rolling forecast')
plt.plot(df_error.iloc[:,0],df_error.iloc[:,2], color='red', label='Normalised RMSE, Static Forecast')
plt.legend()
plt.xlabel("Time of the day", fontsize=20)
plt.ylabel("Energy- KW", fontsize=20)
plt.title('Normalised RMSE for Energy Consumption prediction in the ARIMAX Model', fontsize=20)
plt.xticks(rotation = 90)
plt.show()

In [ ]:
#plot rmse for each time slot
fig, ax = plt.subplots(figsize=(21,6))
plt.plot(df_Rolling_error.iloc[:,0],df_Rolling_error.iloc[:,1], color='green', label='RMSE, Rolling Forecast')
plt.plot(df_error.iloc[:,0],df_error.iloc[:,1], color='red', label='RMSE, Static Forecast')
plt.legend()
plt.xlabel("Time of the day", fontsize=20)
plt.ylabel("Energy- KW", fontsize=20)
plt.title('RMSE for Energy Consumption prediction in ARIMAX Model', fontsize=20)
plt.xticks(rotation = 90)
plt.show()

In [ ]:
max(df_Rolling_error['RMSE']) #shows maximum error in time slots

In [ ]:
min(df_Rolling_error['RMSE']) #shows minimum error in time slots

In [ ]:
average(df_error.iloc[:,1]) #shows average rmse for all time slots in static forecast

In [ ]:
average(df_Rolling_error.iloc[:,1]) #shows average rmse for all time slots in rolling forecast

In [ ]:
df_p_q #the dataframe that shows us the best order of p and q for each time slot

In [ ]:
df= df.sort_index() #sorting the static prediction based on the date and time
df

In [ ]:
df_rolling=df_rolling.sort_index() #sorting the rolling prediction based on the date and time

In [ ]:
#plotting predictions versus real data
fig, ax = plt.subplots(figsize=(21,5))
plt.plot(energy.iloc[17183:,0], color='black',label='Real Energy Consumption- $X_t$')
plt.plot(df.iloc[:,0], color= 'red', alpha=0.7,label=' Predicted Energy Consumption- Static Forecast-'+ ' $\hat{X}$')
plt.plot(df_rolling.iloc[:,0], color='green', label='Predicted Energy Consumption- Rolling forecast-' + ' $\hat{X}$')
#plt.title('Prediction for 1 Week', fontsize=15)
#plt.ylabel('Energy (KWh)')
plt.xlabel('Date',fontsize=15)
plt.legend()
plt.show()

In [ ]:
#calculating mean absolute percentage error in order to compare with other ML models
mape = mean_absolute_percentage_error(energy.iloc[17183:,0], df_rolling.iloc[:,0])*100
print('Mean Absolute Percentage Error for Rolling forecast:', str(mape)+'%')

In [ ]:
energy.iloc[17183:,0]

In [ ]:
df.iloc[:,0]

In [ ]:
mape = mean_absolute_percentage_error(energy.iloc[17183:,0], df.iloc[:,0])*100
print('Mean Absolute Percentage Error for Static forecast:', str(mape)+'%')

In [ ]:
rmse = rmse(energy.iloc[17183:,0], df_rolling.iloc[:,0])
print('RMSE for Rolling forecast:', str(rmse)+'%')

In [ ]:
#creating a data frame with real data, rolling and static predictions for 1 week (24-30 April)
comparing_df= pd.DataFrame(data=energy.iloc[17183:,0], index=energy.iloc[17183:,1].index )
comparing_df['Rolling Energy Prediction Values']= df_rolling.iloc[:,0]
comparing_df['Energy Prediction Values']= df.iloc[:,0]
#residuals for rolling forecast
comparing_df['Rolling difference']= comparing_df['Energy']-comparing_df['Rolling Energy Prediction Values']


In [ ]:
#plotting the whole residuals for rolling forecast
fig, ax = plt.subplots(figsize=(20,5))
plt.plot(comparing_df['Rolling difference'], color= 'black')
plt.title('Residuals for a week prediction')
plt.ylabel('Error (kwh)')
plt.show()


In [ ]:
#plotting the distribution for the whole residuals (rolling forecast)
fig, ax = plt.subplots(figsize=(20,5))

sns.distplot(comparing_df['Rolling difference'], hist=True, kde=True,
             bins=30, color = 'darkblue',
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 4}).set(title= 'Residual Distribution for a week prediction')


In [ ]:
comparing_df.sort_values('Rolling difference', ascending= False)
#box plot to show range of residuals
sns.boxplot(data=comparing_df['Rolling difference']).set(title='Range of all residuals')

In [ ]:
#get other data, price
price= pd.read_csv('Energy Pricing.csv')

In [ ]:
#make the data ready for datetime type
price.Date = pd.to_datetime(price.Date)
price['Week'] = price.Date.dt.week
price['Year'] = price.Date.dt.year
price['Month'] = price.Date.dt.month
price['Day'] = price.Date.dt.day
price['Hour'] = price.Date.dt.hour
price["Min"]=price.Date.dt.minute
price['Month_Year'] = price['Date'].dt.strftime('%b-%Y')
price['Day_Month'] = price['Date'].dt.strftime('%d-%b')
price.index= price['Date']
del price['Date']
price

In [ ]:
#plotting the price and energy together for a one week (last week of april)
fig, ax = plt.subplots(figsize=(20,7))
plt.plot(price.iloc[-168:-1,0], color='gold', label='Price- Pound/MWH')
plt.plot(energy.iloc[17183:,0], color='black', label='$X_t$')
#plt.title('Price and Energy consumption comparission', fontsize=20)
plt.xlabel('Date', fontsize=15)
plt.legend()
plt.show()

# Optimization

In [ ]:
#prepare a data frame which contains data from rolling forecast prediction for a week;
df_rolling['Date']=df_rolling.index
df_rolling.Date = pd.to_datetime(df_rolling.Date)
df_rolling['Month'] = df_rolling.Date.dt.month
df_rolling['Day'] = df_rolling.Date.dt.day
del df_rolling['Date']
df_rolling

In [ ]:
#defining E as the variables to be optimized with a upper and lower bounds
#we have 48 variables each for half an hour (48 time slots)
E1_bounds = (min(df_rolling.iloc[:,0]), max(df_rolling.iloc[:,0]))
E2_bounds = (min(df_rolling.iloc[:,0]), max(df_rolling.iloc[:,0]))
E3_bounds = (min(df_rolling.iloc[:,0]), max(df_rolling.iloc[:,0]))
E4_bounds = (min(df_rolling.iloc[:,0]), max(df_rolling.iloc[:,0]))
E5_bounds = (min(df_rolling.iloc[:,0]), max(df_rolling.iloc[:,0]))
E6_bounds = (min(df_rolling.iloc[:,0]), max(df_rolling.iloc[:,0]))
E7_bounds = (min(df_rolling.iloc[:,0]), max(df_rolling.iloc[:,0]))
E8_bounds = (min(df_rolling.iloc[:,0]), max(df_rolling.iloc[:,0]))
E9_bounds = (min(df_rolling.iloc[:,0]), max(df_rolling.iloc[:,0]))
E10_bounds = (min(df_rolling.iloc[:,0]), max(df_rolling.iloc[:,0]))
E11_bounds = (min(df_rolling.iloc[:,0]), max(df_rolling.iloc[:,0]))
E12_bounds = (min(df_rolling.iloc[:,0]), max(df_rolling.iloc[:,0]))
E13_bounds = (min(df_rolling.iloc[:,0]), max(df_rolling.iloc[:,0]))
E14_bounds = (min(df_rolling.iloc[:,0]), max(df_rolling.iloc[:,0]))
E15_bounds = (min(df_rolling.iloc[:,0]), max(df_rolling.iloc[:,0]))
E16_bounds = (min(df_rolling.iloc[:,0]), max(df_rolling.iloc[:,0]))
E17_bounds = (min(df_rolling.iloc[:,0]), max(df_rolling.iloc[:,0]))
E18_bounds = (min(df_rolling.iloc[:,0]), max(df_rolling.iloc[:,0]))
E19_bounds = (min(df_rolling.iloc[:,0]), max(df_rolling.iloc[:,0]))
E20_bounds = (min(df_rolling.iloc[:,0]), max(df_rolling.iloc[:,0]))
E21_bounds = (min(df_rolling.iloc[:,0]), max(df_rolling.iloc[:,0]))
E22_bounds = (min(df_rolling.iloc[:,0]), max(df_rolling.iloc[:,0]))
E23_bounds = (min(df_rolling.iloc[:,0]), max(df_rolling.iloc[:,0]))
E24_bounds = (min(df_rolling.iloc[:,0]), max(df_rolling.iloc[:,0]))
E25_bounds = (min(df_rolling.iloc[:,0]), max(df_rolling.iloc[:,0]))
E26_bounds = (min(df_rolling.iloc[:,0]), max(df_rolling.iloc[:,0]))
E27_bounds = (min(df_rolling.iloc[:,0]), max(df_rolling.iloc[:,0]))
E28_bounds = (min(df_rolling.iloc[:,0]), max(df_rolling.iloc[:,0]))
E29_bounds = (min(df_rolling.iloc[:,0]), max(df_rolling.iloc[:,0]))
E30_bounds = (min(df_rolling.iloc[:,0]), max(df_rolling.iloc[:,0]))
E31_bounds = (min(df_rolling.iloc[:,0]), max(df_rolling.iloc[:,0]))
E32_bounds = (min(df_rolling.iloc[:,0]), max(df_rolling.iloc[:,0]))
E33_bounds = (min(df_rolling.iloc[:,0]), max(df_rolling.iloc[:,0]))
E34_bounds = (min(df_rolling.iloc[:,0]), max(df_rolling.iloc[:,0]))
E35_bounds = (min(df_rolling.iloc[:,0]), max(df_rolling.iloc[:,0]))
E36_bounds = (min(df_rolling.iloc[:,0]), max(df_rolling.iloc[:,0]))
E37_bounds = (min(df_rolling.iloc[:,0]), max(df_rolling.iloc[:,0]))
E38_bounds = (min(df_rolling.iloc[:,0]), max(df_rolling.iloc[:,0]))
E39_bounds = (min(df_rolling.iloc[:,0]), max(df_rolling.iloc[:,0]))
E40_bounds = (min(df_rolling.iloc[:,0]), max(df_rolling.iloc[:,0]))
E41_bounds = (min(df_rolling.iloc[:,0]), max(df_rolling.iloc[:,0]))
E42_bounds = (min(df_rolling.iloc[:,0]), max(df_rolling.iloc[:,0]))
E43_bounds = (min(df_rolling.iloc[:,0]), max(df_rolling.iloc[:,0]))
E44_bounds = (min(df_rolling.iloc[:,0]), max(df_rolling.iloc[:,0]))
E45_bounds = (min(df_rolling.iloc[:,0]), max(df_rolling.iloc[:,0]))
E46_bounds = (min(df_rolling.iloc[:,0]), max(df_rolling.iloc[:,0]))
E47_bounds = (min(df_rolling.iloc[:,0]), max(df_rolling.iloc[:,0]))
E48_bounds = (min(df_rolling.iloc[:,0]), max(df_rolling.iloc[:,0]))

In [ ]:
#predicting for 24th of April
predicted_energy= df_rolling.loc[(df_rolling['Month'] == 4)&(df_rolling['Day'] == 24)] #get the data for 24th
b=price.loc[(price['Month'] == 4)&(price['Day'] == 24)] #get price for 24th
predicted_energy['price']=b.iloc[:,0]
for i in range (0,48):
    if i%2==1:
        predicted_energy['price'][i]=predicted_energy['price'][i-1] #define price values for half hours
predicted_energy.columns = ['Predicted_Energy','Month', 'Day', 'Price']

f=[] #price of total enery used before optimization
for i in range(0,48):
    f.append(predicted_energy['Predicted_Energy'][i]*predicted_energy['Price'][i]*0.001)



# declare coefficients of the objective function which is price
c = list(predicted_energy['Price']*0.001)

# declare the inequality constraint matrix
A = [[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
     [0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
     [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0],
     [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1]]

# declare the inequality constraint vector
b = [-sum(predicted_energy.iloc[0:12,0]), -sum(predicted_energy.iloc[12:24,0]), -sum(predicted_energy.iloc[24:36,0]),-sum(predicted_energy.iloc[36:48,0])]

# solve
results = linprog(c=c, A_ub=A, b_ub=b, bounds=[E1_bounds, E2_bounds,E3_bounds, E4_bounds,
                                              E5_bounds, E6_bounds,E7_bounds, E8_bounds,
                                              E9_bounds, E10_bounds,E11_bounds, E12_bounds,
                                              E13_bounds, E14_bounds,E15_bounds, E16_bounds,
                                              E17_bounds, E18_bounds,E19_bounds, E20_bounds,
                                              E21_bounds, E22_bounds,E23_bounds, E24_bounds,
                                              E25_bounds, E26_bounds,E27_bounds, E28_bounds,
                                              E29_bounds, E30_bounds,E31_bounds, E32_bounds,
                                              E33_bounds, E34_bounds,E35_bounds, E36_bounds,
                                              E37_bounds, E38_bounds,E39_bounds, E40_bounds,
                                              E41_bounds, E42_bounds,E43_bounds, E44_bounds,
                                              E45_bounds, E46_bounds,E47_bounds, E48_bounds], method='simplex')

# print results
if results.status == 0: print(f'The solution is optimal.')
print(f'Objective value: z* = {results.fun}')

Optimized_24=[]
for i in range(0,48):
    Optimized_24.append(results.x[i])
Optimized_df= pd.DataFrame(data=Optimized_24, index=predicted_energy.index) #make a dataframe contains the optimized values

#optimized value
print('Percent of saved money:',((sum(f)-results.fun)/sum(f))*100)
z_star=[]
z_star.append(results.fun)
F=[]
F.append(sum(f))

In [ ]:
#predicting for 25th of April
predicted_energy= df_rolling.loc[(df_rolling['Month'] == 4)&(df_rolling['Day'] == 25)]
b=price.loc[(price['Month'] == 4)&(price['Day'] == 25)]
predicted_energy['price']=b.iloc[:,0]
for i in range (0,48):
    if i%2==1:
        predicted_energy['price'][i]=predicted_energy['price'][i-1]
predicted_energy.columns = ['Predicted_Energy','Month', 'Day', 'Price']

f=[]
for i in range(0,48):
    f.append(predicted_energy['Predicted_Energy'][i]*predicted_energy['Price'][i]*0.001)



# declare coefficients of the objective function
c = list(predicted_energy['Price']*0.001)

# declare the inequality constraint matrix
A = [[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
     [0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
     [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0],
     [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1]]

# declare the inequality constraint vector
b = [-sum(predicted_energy.iloc[0:12,0]), -sum(predicted_energy.iloc[12:24,0]), -sum(predicted_energy.iloc[24:36,0]),-sum(predicted_energy.iloc[36:48,0])]

# solve
results = linprog(c=c, A_ub=A, b_ub=b, bounds=[E1_bounds, E2_bounds,E3_bounds, E4_bounds,
                                              E5_bounds, E6_bounds,E7_bounds, E8_bounds,
                                              E9_bounds, E10_bounds,E11_bounds, E12_bounds,
                                              E13_bounds, E14_bounds,E15_bounds, E16_bounds,
                                              E17_bounds, E18_bounds,E19_bounds, E20_bounds,
                                              E21_bounds, E22_bounds,E23_bounds, E24_bounds,
                                              E25_bounds, E26_bounds,E27_bounds, E28_bounds,
                                              E29_bounds, E30_bounds,E31_bounds, E32_bounds,
                                              E33_bounds, E34_bounds,E35_bounds, E36_bounds,
                                              E37_bounds, E38_bounds,E39_bounds, E40_bounds,
                                              E41_bounds, E42_bounds,E43_bounds, E44_bounds,
                                              E45_bounds, E46_bounds,E47_bounds, E48_bounds], method='simplex')

# print results
if results.status == 0: print(f'The solution is optimal.')
print(f'Objective value: z* = {results.fun}')

Optimized_24=[]
for i in range(0,48):
    Optimized_24.append(results.x[i])
Optimized_df= Optimized_df.append(pd.DataFrame(data=Optimized_24, index=predicted_energy.index))

#optimized value
print('Percent of saved money:',((sum(f)-results.fun)/sum(f))*100)
z_star.append(results.fun)
F.append(sum(f))

In [ ]:
#predicting for 26th of April
predicted_energy= df_rolling.loc[(df_rolling['Month'] == 4)&(df_rolling['Day'] == 26)]
b=price.loc[(price['Month'] == 4)&(price['Day'] == 26)]
predicted_energy['price']=b.iloc[:,0]
for i in range (0,48):
    if i%2==1:
        predicted_energy['price'][i]=predicted_energy['price'][i-1]
predicted_energy.columns = ['Predicted_Energy','Month', 'Day', 'Price']

f=[]
for i in range(0,48):
    f.append(predicted_energy['Predicted_Energy'][i]*predicted_energy['Price'][i]*0.001)



# declare coefficients of the objective function
c = list(predicted_energy['Price']*0.001)

# declare the inequality constraint matrix
A = [[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
     [0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
     [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0],
     [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1]]

# declare the inequality constraint vector
b = [-sum(predicted_energy.iloc[0:12,0]), -sum(predicted_energy.iloc[12:24,0]), -sum(predicted_energy.iloc[24:36,0]),-sum(predicted_energy.iloc[36:48,0])]

# solve
results = linprog(c=c, A_ub=A, b_ub=b, bounds=[E1_bounds, E2_bounds,E3_bounds, E4_bounds,
                                              E5_bounds, E6_bounds,E7_bounds, E8_bounds,
                                              E9_bounds, E10_bounds,E11_bounds, E12_bounds,
                                              E13_bounds, E14_bounds,E15_bounds, E16_bounds,
                                              E17_bounds, E18_bounds,E19_bounds, E20_bounds,
                                              E21_bounds, E22_bounds,E23_bounds, E24_bounds,
                                              E25_bounds, E26_bounds,E27_bounds, E28_bounds,
                                              E29_bounds, E30_bounds,E31_bounds, E32_bounds,
                                              E33_bounds, E34_bounds,E35_bounds, E36_bounds,
                                              E37_bounds, E38_bounds,E39_bounds, E40_bounds,
                                              E41_bounds, E42_bounds,E43_bounds, E44_bounds,
                                              E45_bounds, E46_bounds,E47_bounds, E48_bounds], method='simplex')

# print results
if results.status == 0: print(f'The solution is optimal.')
print(f'Objective value: z* = {results.fun}')

Optimized_24=[]
for i in range(0,48):
    Optimized_24.append(results.x[i])
Optimized_df= Optimized_df.append(pd.DataFrame(data=Optimized_24, index=predicted_energy.index))

#optimized value
print('Percent of saved money:',((sum(f)-results.fun)/sum(f))*100)
z_star.append(results.fun)
F.append(sum(f))

In [ ]:
#predicting for 27th of April
predicted_energy= df_rolling.loc[(df_rolling['Month'] == 4)&(df_rolling['Day'] == 27)]
b=price.loc[(price['Month'] == 4)&(price['Day'] == 27)]
predicted_energy['price']=b.iloc[:,0]
for i in range (0,48):
    if i%2==1:
        predicted_energy['price'][i]=predicted_energy['price'][i-1]
predicted_energy.columns = ['Predicted_Energy','Month', 'Day', 'Price']

f=[]
for i in range(0,48):
    f.append(predicted_energy['Predicted_Energy'][i]*predicted_energy['Price'][i]*0.001)



# declare coefficients of the objective function
c = list(predicted_energy['Price']*0.001)

# declare the inequality constraint matrix
A = [[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
     [0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
     [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0],
     [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1]]

# declare the inequality constraint vector
b = [-sum(predicted_energy.iloc[0:12,0]), -sum(predicted_energy.iloc[12:24,0]), -sum(predicted_energy.iloc[24:36,0]),-sum(predicted_energy.iloc[36:48,0])]

# solve
results = linprog(c=c, A_ub=A, b_ub=b, bounds=[E1_bounds, E2_bounds,E3_bounds, E4_bounds,
                                              E5_bounds, E6_bounds,E7_bounds, E8_bounds,
                                              E9_bounds, E10_bounds,E11_bounds, E12_bounds,
                                              E13_bounds, E14_bounds,E15_bounds, E16_bounds,
                                              E17_bounds, E18_bounds,E19_bounds, E20_bounds,
                                              E21_bounds, E22_bounds,E23_bounds, E24_bounds,
                                              E25_bounds, E26_bounds,E27_bounds, E28_bounds,
                                              E29_bounds, E30_bounds,E31_bounds, E32_bounds,
                                              E33_bounds, E34_bounds,E35_bounds, E36_bounds,
                                              E37_bounds, E38_bounds,E39_bounds, E40_bounds,
                                              E41_bounds, E42_bounds,E43_bounds, E44_bounds,
                                              E45_bounds, E46_bounds,E47_bounds, E48_bounds], method='simplex')

# print results
if results.status == 0: print(f'The solution is optimal.')
print(f'Objective value: z* = {results.fun}')

Optimized_24=[]
for i in range(0,48):
    Optimized_24.append(results.x[i])
Optimized_df= Optimized_df.append(pd.DataFrame(data=Optimized_24, index=predicted_energy.index))

#optimized value
print('Percent of saved money:',((sum(f)-results.fun)/sum(f))*100)
z_star.append(results.fun)
F.append(sum(f))

In [ ]:
#predicting for 28th of April
predicted_energy= df_rolling.loc[(df_rolling['Month'] == 4)&(df_rolling['Day'] == 28)]
b=price.loc[(price['Month'] == 4)&(price['Day'] == 28)]
predicted_energy['price']=b.iloc[:,0]
for i in range (0,48):
    if i%2==1:
        predicted_energy['price'][i]=predicted_energy['price'][i-1]
predicted_energy.columns = ['Predicted_Energy','Month', 'Day', 'Price']

f=[]
for i in range(0,48):
    f.append(predicted_energy['Predicted_Energy'][i]*predicted_energy['Price'][i]*0.001)



# declare coefficients of the objective function
c = list(predicted_energy['Price']*0.001)

# declare the inequality constraint matrix
A = [[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
     [0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
     [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0],
     [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1]]

# declare the inequality constraint vector
b = [-sum(predicted_energy.iloc[0:12,0]), -sum(predicted_energy.iloc[12:24,0]), -sum(predicted_energy.iloc[24:36,0]),-sum(predicted_energy.iloc[36:48,0])]

# solve
results = linprog(c=c, A_ub=A, b_ub=b, bounds=[E1_bounds, E2_bounds,E3_bounds, E4_bounds,
                                              E5_bounds, E6_bounds,E7_bounds, E8_bounds,
                                              E9_bounds, E10_bounds,E11_bounds, E12_bounds,
                                              E13_bounds, E14_bounds,E15_bounds, E16_bounds,
                                              E17_bounds, E18_bounds,E19_bounds, E20_bounds,
                                              E21_bounds, E22_bounds,E23_bounds, E24_bounds,
                                              E25_bounds, E26_bounds,E27_bounds, E28_bounds,
                                              E29_bounds, E30_bounds,E31_bounds, E32_bounds,
                                              E33_bounds, E34_bounds,E35_bounds, E36_bounds,
                                              E37_bounds, E38_bounds,E39_bounds, E40_bounds,
                                              E41_bounds, E42_bounds,E43_bounds, E44_bounds,
                                              E45_bounds, E46_bounds,E47_bounds, E48_bounds], method='simplex')

# print results
if results.status == 0: print(f'The solution is optimal.')
print(f'Objective value: z* = {results.fun}')

Optimized_24=[]
for i in range(0,48):
    Optimized_24.append(results.x[i])
Optimized_df= Optimized_df.append(pd.DataFrame(data=Optimized_24, index=predicted_energy.index))

#optimized value
print('Percent of saved money:',((sum(f)-results.fun)/sum(f))*100)
z_star.append(results.fun)
F.append(sum(f))

In [ ]:
#predicting for 29th of April
predicted_energy= df_rolling.loc[(df_rolling['Month'] == 4)&(df_rolling['Day'] == 29)]
b=price.loc[(price['Month'] == 4)&(price['Day'] == 29)]
predicted_energy['price']=b.iloc[:,0]
for i in range (0,48):
    if i%2==1:
        predicted_energy['price'][i]=predicted_energy['price'][i-1]
predicted_energy.columns = ['Predicted_Energy','Month', 'Day', 'Price']

f=[]
for i in range(0,48):
    f.append(predicted_energy['Predicted_Energy'][i]*predicted_energy['Price'][i]*0.001)



# declare coefficients of the objective function
c = list(predicted_energy['Price']*0.001)

# declare the inequality constraint matrix
A = [[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
     [0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
     [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0],
     [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1]]

# declare the inequality constraint vector
b = [-sum(predicted_energy.iloc[0:12,0]), -sum(predicted_energy.iloc[12:24,0]), -sum(predicted_energy.iloc[24:36,0]),-sum(predicted_energy.iloc[36:48,0])]

# solve
results = linprog(c=c, A_ub=A, b_ub=b, bounds=[E1_bounds, E2_bounds,E3_bounds, E4_bounds,
                                              E5_bounds, E6_bounds,E7_bounds, E8_bounds,
                                              E9_bounds, E10_bounds,E11_bounds, E12_bounds,
                                              E13_bounds, E14_bounds,E15_bounds, E16_bounds,
                                              E17_bounds, E18_bounds,E19_bounds, E20_bounds,
                                              E21_bounds, E22_bounds,E23_bounds, E24_bounds,
                                              E25_bounds, E26_bounds,E27_bounds, E28_bounds,
                                              E29_bounds, E30_bounds,E31_bounds, E32_bounds,
                                              E33_bounds, E34_bounds,E35_bounds, E36_bounds,
                                              E37_bounds, E38_bounds,E39_bounds, E40_bounds,
                                              E41_bounds, E42_bounds,E43_bounds, E44_bounds,
                                              E45_bounds, E46_bounds,E47_bounds, E48_bounds], method='simplex')

# print results
if results.status == 0: print(f'The solution is optimal.')
print(f'Objective value: z* = {results.fun}')

Optimized_24=[]
for i in range(0,48):
    Optimized_24.append(results.x[i])
Optimized_df= Optimized_df.append(pd.DataFrame(data=Optimized_24, index=predicted_energy.index))

#optimized value
print('Percent of saved money:',((sum(f)-results.fun)/sum(f))*100)
z_star.append(results.fun)
F.append(sum(f))

In [ ]:
#predicting for 30th of April
predicted_energy= df_rolling.loc[(df_rolling['Month'] == 4)&(df_rolling['Day'] == 30)]
b=price.loc[(price['Month'] == 4)&(price['Day'] == 30)]
predicted_energy['price']=b.iloc[:,0]
for i in range (0,48):
    if i%2==1:
        predicted_energy['price'][i]=predicted_energy['price'][i-1]
predicted_energy.columns = ['Predicted_Energy','Month', 'Day', 'Price']

f=[]
for i in range(0,48):
    f.append(predicted_energy['Predicted_Energy'][i]*predicted_energy['Price'][i]*0.001)



# declare coefficients of the objective function
c = list(predicted_energy['Price']*0.001)

# declare the inequality constraint matrix
A = [[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
     [0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
     [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0],
     [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1]]

# declare the inequality constraint vector
b = [-sum(predicted_energy.iloc[0:12,0]), -sum(predicted_energy.iloc[12:24,0]), -sum(predicted_energy.iloc[24:36,0]),-sum(predicted_energy.iloc[36:48,0])]

# solve
results = linprog(c=c, A_ub=A, b_ub=b, bounds=[E1_bounds, E2_bounds,E3_bounds, E4_bounds,
                                              E5_bounds, E6_bounds,E7_bounds, E8_bounds,
                                              E9_bounds, E10_bounds,E11_bounds, E12_bounds,
                                              E13_bounds, E14_bounds,E15_bounds, E16_bounds,
                                              E17_bounds, E18_bounds,E19_bounds, E20_bounds,
                                              E21_bounds, E22_bounds,E23_bounds, E24_bounds,
                                              E25_bounds, E26_bounds,E27_bounds, E28_bounds,
                                              E29_bounds, E30_bounds,E31_bounds, E32_bounds,
                                              E33_bounds, E34_bounds,E35_bounds, E36_bounds,
                                              E37_bounds, E38_bounds,E39_bounds, E40_bounds,
                                              E41_bounds, E42_bounds,E43_bounds, E44_bounds,
                                              E45_bounds, E46_bounds,E47_bounds, E48_bounds], method='simplex')

# print results
if results.status == 0: print(f'The solution is optimal.')
print(f'Objective value: z* = {results.fun}')

Optimized_24=[]
for i in range(0,48):
    Optimized_24.append(results.x[i])
Optimized_df= Optimized_df.append(pd.DataFrame(data=Optimized_24, index=predicted_energy.index))

#optimized value
print('Percent of saved money:',((sum(f)-results.fun)/sum(f))*100)
z_star.append(results.fun)
F.append(sum(f))

In [ ]:
Optimized_df.columns=['Optimized Energy']
Optimized_df

In [ ]:
sum(Optimized_df['Optimized Energy']) #make sure optimized total energy is equal to predicted one

In [ ]:
sum(df_rolling.iloc[:,0])

In [ ]:
#plotting optimized pattern of energy consumption versus before optimization (rolling forecast)
df3=price.loc[(price['Month'] == 4)]
fig, ax = plt.subplots(figsize=(21,5))
plt.plot(df_rolling.iloc[:,0], 'g--', label='Rolling forecast of energy consumption (kw) - Before Optimiztion')
plt.plot(Optimized_df.iloc[:,0], color='brown', label='Optimized Energy Pattern (kw)')
#plt.plot(price.iloc[-168:-1,0], color='gold', label='Price (Pounds/MWH)')
plt.xlabel('Date: Last week of April', fontsize=15)
#plt.ylabel('kwh',fontsize=15)
#plt.title('Before and After Optimization', fontsize=20)
plt.legend(fontsize=12)
plt.show()

In [ ]:
Time= ['12:00','12:30','13:00','13:30','14:00','14:30','15:00','15:30','16:00','16:30','17:00','17:30','18:00','18:30','19:00','19:30','20:00','20:30','21:00','21:30','22:00','22:30','23:00','23:30' ]

In [ ]:
Time=['12:00', '14:00', '16:00', '18:00', '20:00', '22:00']

In [ ]:
#plotting energy pattern before and after optimization for 12 hours
fig, ax = plt.subplots(figsize=(21,5))
plt.plot(df_rolling.iloc[-24:-1,0], 'g--', label='Required Energy- Before Optimiztion, kw')
plt.plot(Optimized_df.iloc[-24:-1,0], color='brown', label='Optimized Energy Pattern, kw')
plt.plot(price.iloc[-12:-1,0], color='gold', label='Price (Pounds/MWH)')
#plt.title('Energy consumption pattern Before and After Optimization and Energy Tariff for 12 Hours', fontsize=20)
ax.set_xticklabels(Time, rotation='vertical', fontsize=12)
plt.xlabel('Time', fontsize=15)
plt.legend()
plt.show()

In [ ]:
#comparing cost before and after optimization
fig, ax = plt.subplots(figsize=(21,5))
plt.plot(range(24,31),F,'g--', label='Cost BEFORE optimization')
plt.plot(range(24,31),z_star,color='brown', label='Cost AFTER optimization')
plt.xlabel('Date in April 2021', fontsize=15)
plt.ylabel('Daily energy cost (£)', fontsize=15)
#plt.title('Cost comparison', fontsize=20)
plt.legend()
plt.show()

In [ ]:
print('Total percentage of savings:',str(((sum(F)-sum(z_star))/sum(F))*100)+'%')

In [ ]:
print('Total amount of savings:', str(sum(F)-sum(z_star))+' pounds')

In [ ]:
print('Total energy used after optimization: ',sum(Optimized_df.iloc[:,0]))

In [ ]:
print('Total energy used before optimization: ',sum(df_rolling.iloc[:,0]))

In [ ]:
sum(F)

In [ ]:
sum(z_star)

# Check proposed time series model for two weeks

In [ ]:
df2=energy.loc[(energy['Hour'] == 0) & (energy['Min'] == 0)]
df2=df2.iloc[:,:2]
df2

In [ ]:
#for 00:00
from statsmodels.tsa.statespace.sarimax import SARIMAX
AIC_orders=[]

# Loop over p values from 0-20
for q in range(4):
  # Loop over q values from 0-20
    for p in range(7):
      	# create and fit ARIMA(p,q) model (d=1 because difference of data is stationary (adfuller test)); traning data is
        #given to check the aic
        model = SARIMAX(df2.iloc[0:350,0],exog=df2.iloc[0:350,1], order=(p,1,q),enforce_invertibility=False, enforce_stationarity=False)
        results = model.fit()

        # Append order and results tuple
        AIC_orders.append((p,q,results.aic))

# DataFrame from AIC_orders
order_df = pd.DataFrame(AIC_orders,
                        columns=['p','q','AIC'])

order_df=order_df.sort_values('AIC')
order_df = order_df.reset_index()
del order_df['index']

kn = KneeLocator(range(0,28),order_df['AIC'], curve='convex', direction='increasing') #finding the elbow
h=kn.knee


data = [{'time':'0','p':order_df['p'][h], 'q':order_df['q'][h]}]

df_p_q = pd.DataFrame(data, columns=['time','p','q'])

#defining the model based on the optimum p and q that AIC returned.
model= SARIMAX(df2.iloc[0:350,0],exog=df2.iloc[0:350,1],order=(order_df['p'][h],1,order_df['q'][h]), enforce_invertibility=False, enforce_stationarity=False)
results= model.fit()
predictions= results.predict(start =350, end=363, exog= df2.iloc[350:,1])
predictions.index= df2.iloc[350:,0].index
df=pd.DataFrame(data=predictions)

from statsmodels.tools.eval_measures import rmse
Error_data = [{'time':'0','RMSE':rmse(predictions,df2.iloc[350:,0]), 'Normalized RMSE':rmse(predictions,df2.iloc[350:,0])/average(df2.iloc[350:,0])}]
df_error= pd.DataFrame(Error_data, columns=['time','RMSE','Normalized RMSE'])

# Rolling forecast

train, test = df2.iloc[0:350,0].values, df2.iloc[350:,0].values
train_exo, test_exo= df2.iloc[0:350,1].values, df2.iloc[350:,1].values
history = [x for x in train]
history_exo = [y for y in train_exo]
predictions=list()
for t in range(len(test)):
    model = SARIMAX(history,exog= history_exo,order=(order_df['p'][h],1,order_df['q'][h]),enforce_invertibility=False, enforce_stationarity=False)
    fit_model = model.fit()
    out= fit_model.forecast(exog= test_exo[t])
    yhat = out[0]
    predictions.append(yhat)
    J = test[t]
    s= test_exo[t]
    history.append(J)
    history_exo.append(s)

df_rolling=pd.DataFrame(data= predictions, index= df2.iloc[350:,0].index)  #adding predicted values to previous predictions

Error_Rolling_data = [{'time':'0','RMSE':rmse(predictions,df2.iloc[350:,0]), 'Normalized RMSE':rmse(predictions,df2.iloc[350:,0])/average(df2.iloc[350:,0])}]
df_Rolling_error= pd.DataFrame(Error_data, columns=['time','RMSE','Normalized RMSE'])



In [ ]:
#for 00:30
df2=energy.loc[(energy['Hour'] == 0) & (energy['Min'] == 30)]
df2=df2.iloc[:,:2]


AIC_orders=[]

    # Loop over p values from 0-20
for q in range(4):
      # Loop over q values from 0-20
    for p in range(7):
            # create and fit ARIMA(p,q) model (d=1 because difference of data is stationary (adfuller test)); traning data is
            #given to check the aic
        model = SARIMAX(df2.iloc[0:351,0],exog=df2.iloc[0:351,1],order=(p,1,q),enforce_invertibility=False, enforce_stationarity=False)
        results = model.fit()

            # Append order and results tuple
        AIC_orders.append((p,q,results.aic))

    # DataFrame from AIC_orders
order_df = pd.DataFrame(AIC_orders,
                            columns=['p','q','AIC'])
    #sorting based on the minimum values of AIC
order_df=order_df.sort_values('AIC')
order_df = order_df.reset_index()
del order_df['index']

kn = KneeLocator(range(0,28),order_df['AIC'], curve='convex', direction='increasing') #finding the elbow
h=kn.knee

data = ['00:30',order_df['p'][h], order_df['q'][h]]
df_p_q.loc[len(df_p_q)]=data

    #defining the model based on the optimum p and q that AIC returned.
model= SARIMAX(df2.iloc[0:351,0],exog=df2.iloc[0:351,1], order=(order_df['p'][h],1,order_df['q'][h]), enforce_invertibility=False, enforce_stationarity=False)
results= model.fit()
predictions= results.predict(start =351, end=364,exog= df2.iloc[351:,1])
predictions.index= df2.iloc[351:,0].index
df=df.append(pd.DataFrame(data=predictions))


Error_data=['00:30',rmse(predictions,df2.iloc[351:,0]), rmse(predictions,df2.iloc[351:,0])/average(df2.iloc[351:,0])]
df_error.loc[len(df_error)]=Error_data


    #rolling:

train, test = df2.iloc[0:351,0].values, df2.iloc[351:,0].values
train_exo, test_exo= df2.iloc[0:351,1].values, df2.iloc[351:,1].values
history = [x for x in train]
history_exo = [y for y in train_exo]
predictions=list()
for t in range(len(test)):
    model = SARIMAX(history,exog= history_exo,order=(order_df['p'][h],1,order_df['q'][h]),enforce_invertibility=False, enforce_stationarity=False)
    fit_model = model.fit()
    out= fit_model.forecast(exog= test_exo[t])
    yhat = out[0]
    predictions.append(yhat)
    J = test[t]
    s= test_exo[t]
    history.append(J)
    history_exo.append(s)
df_rolling= df_rolling.append(pd.DataFrame(data= predictions, index= df2.iloc[351:,0].index))   #adding new predictions to previous hours values

Error_Rolling_data=['00:30',rmse(predictions,df2.iloc[351,0]), rmse(predictions,df2.iloc[351,0])/average(df2.iloc[351,0])]
df_Rolling_error.loc[len(df_Rolling_error)]=Error_Rolling_data


In [ ]:
for i in range(1,24):
    for j in range(0,31,30):
        df2=energy.loc[(energy['Hour'] == i) & (energy['Min'] == j) ]
        df2=df2.iloc[:,:2]
        AIC_orders=[]

        # Loop over p values from 0-20
        for q in range(4):
          # Loop over q values from 0-20
            for p in range(7):
                # create and fit ARIMA(p,q) model (d=1 because difference of data is stationary (adfuller test)); traning data is
                #given to check the aic
                model = SARIMAX(df2.iloc[0:351,0],exog=df2.iloc[0:351,1],order=(p,1,q),enforce_invertibility=False, enforce_stationarity=False)
                results = model.fit()

                # Append order and results tuple
                AIC_orders.append((p,q,results.aic))

        # DataFrame from AIC_orders
        order_df = pd.DataFrame(AIC_orders,
                                columns=['p','q','AIC'])
        #sorting based on the minimum values of AIC
        order_df=order_df.sort_values('AIC')
        order_df = order_df.reset_index()
        del order_df['index']
        kn = KneeLocator(range(0,28),order_df['AIC'], curve='convex', direction='increasing') #finding the elbow
        h=kn.knee

        data = [str(i)+':'+str(j),order_df['p'][h], order_df['q'][h]]
        df_p_q.loc[len(df_p_q)]=data


        #defining a list to collect predicted values.
        #defining the model based on the optimum p and q that AIC returned.
        model= SARIMAX(df2.iloc[0:351,0],exog=df2.iloc[0:351,1], order=(order_df['p'][h],1,order_df['q'][h]), enforce_invertibility=False, enforce_stationarity=False)
        results= model.fit()
        predictions= results.predict(start =351, end=364,exog= df2.iloc[351:,1])
        predictions.index= df2.iloc[351:,0].index
        df=df.append(pd.DataFrame(data=predictions))

        Error_data=[str(i)+':'+str(j),rmse(predictions,df2.iloc[351:,0]), rmse(predictions,df2.iloc[351:,0])/average(df2.iloc[351:,0])]
        df_error.loc[len(df_error)]=Error_data

        # Rolling forecast

        train, test = df2.iloc[0:351,0].values, df2.iloc[351:,0].values
        train_exo, test_exo= df2.iloc[0:351,1].values, df2.iloc[351:,1].values
        history = [x for x in train]
        history_exo = [y for y in train_exo]
        predictions=list()
        for t in range(len(test)):
            model = SARIMAX(history,exog= history_exo,order=(order_df['p'][h],1,order_df['q'][h]),enforce_invertibility=False, enforce_stationarity=False)
            fit_model = model.fit()
            out= fit_model.forecast(exog= test_exo[t])
            yhat = out[0]
            predictions.append(yhat)
            J = test[t]
            s= test_exo[t]
            history.append(J)
            history_exo.append(s)
        df_rolling= df_rolling.append(pd.DataFrame(data= predictions, index=df2.iloc[351:,0].index))  #adding new predictions to previous hours values

        Error_Rolling_data=[str(i)+':'+str(j),rmse(predictions,df2.iloc[351:,0]), rmse(predictions,df2.iloc[351:,0])/average(df2.iloc[351:,0])]
        df_Rolling_error.loc[len(df_Rolling_error)]=Error_Rolling_data



In [ ]:
df= df.sort_index()
df_rolling=df_rolling.sort_index()

In [ ]:
fig, ax = plt.subplots(figsize=(21,5))
plt.plot(energy.iloc[16847:,0], color='black',label='Real Energy Consumption- $X_t$')
plt.plot(df.iloc[:,0], color= 'red', alpha=0.7,label='Predicted Energy Consumption- Static Forecast-'+ ' $\hat{X}$')
plt.plot(df_rolling.iloc[:,0], color='green', label='Predicted Energy Consumption- rolling forecast' + ' $\hat{X}$')

plt.title('Prediction for 2 Weeks', fontsize=15)
#plt.ylabel('kwh')
plt.xlabel('Date',fontsize=15)
plt.legend()
plt.show()

In [ ]:
#calculating mean absolute percentage error in order to compare with other ML models
mape = mean_absolute_percentage_error(energy.iloc[16847:,0], df_rolling.iloc[:,0])*100
print('Mean Absolute Percentage Error for Rolling forecast:', str(mape)+'%')

In [ ]:
mape = mean_absolute_percentage_error(energy.iloc[16847:,0], df.iloc[:,0])*100
print('Mean Absolute Percentage Error for Static forecast:', str(mape)+'%')

In [ ]:
average(df_error.iloc[:,1])

In [ ]:
average(df_Rolling_error.iloc[:,1])

# Checking model for 5 weeks

In [ ]:
#Time: 00:00 (because for this time we have less data for training, it is out of the loop and is done separately)
df2=energy.loc[(energy['Hour'] == 0) & (energy['Min'] == 0)]
df2=df2.iloc[:,:2]
#for 00:00
AIC_orders=[]

# Loop over p values from 0-3
for q in range(4):
  # Loop over q values from 0-6
    for p in range(7):
      	# create and fit ARIMA(p,q) model (d=1 because difference of data is stationary (adfuller test)); traning data is
        #given to check the aic
        model = SARIMAX(df2.iloc[0:329,0],exog=df2.iloc[0:329,1], order=(p,1,q),enforce_invertibility=False, enforce_stationarity=False)
        results = model.fit()

        # Append order and results tuple
        AIC_orders.append((p,q,results.aic))

# DataFrame from AIC_orders
order_df = pd.DataFrame(AIC_orders,
                        columns=['p','q','AIC'])

order_df=order_df.sort_values('AIC')
order_df = order_df.reset_index()
del order_df['index']

kn = KneeLocator(range(0,28),order_df['AIC'], curve='convex', direction='increasing') #finding the elbow
h=kn.knee


data = [{'time':'0','p':order_df['p'][h], 'q':order_df['q'][h]}]

df_p_q = pd.DataFrame(data, columns=['time','p','q'])


#defining the model based on the optimum p and q that AIC returned.
model= SARIMAX(df2.iloc[0:329,0],exog=df2.iloc[0:329,1],order=(order_df['p'][h],1,order_df['q'][h]), enforce_invertibility=False, enforce_stationarity=False)
results= model.fit()

predictions= results.predict(start =329, end=363, exog= df2.iloc[329:,1])
predictions.index= df2.iloc[329:,0].index
df=pd.DataFrame(data=predictions)

Error_data = [{'time':'0','RMSE':rmse(predictions,df2.iloc[329:,0]), 'Normalized RMSE':rmse(predictions,df2.iloc[329:,0])/average(df2.iloc[329:,0])}]
df_error= pd.DataFrame(Error_data, columns=['time','RMSE','Normalized RMSE'])
print('00:00-static')
results.plot_diagnostics(figsize=(21,12))
plt.show()
print(results.summary())
# Rolling forecast

train, test = df2.iloc[0:329,0].values, df2.iloc[329:,0].values
train_exo, test_exo= df2.iloc[0:329,1].values, df2.iloc[329:,1].values
history = [x for x in train]
history_exo = [y for y in train_exo]
predictions=list()
for t in range(len(test)):
    model = SARIMAX(history,exog= history_exo,order=(order_df['p'][h],1,order_df['q'][h]),enforce_invertibility=False, enforce_stationarity=False)
    fit_model = model.fit()
    out= fit_model.forecast(exog= test_exo[t])
    yhat = out[0]
    predictions.append(yhat)
    J = test[t]
    s= test_exo[t]
    history.append(J)
    history_exo.append(s)

df_rolling=pd.DataFrame(data= predictions, index= df2.iloc[329:,0].index)  #adding predicted values to previous predictions

Error_Rolling_data = [{'time':'0','RMSE':rmse(predictions,df2.iloc[329:,0]), 'Normalized RMSE':rmse(predictions,df2.iloc[329:,0])/average(df2.iloc[329:,0])}]
df_Rolling_error= pd.DataFrame(Error_data, columns=['time','RMSE','Normalized RMSE'])
print('00:00-rolling')
fit_model.plot_diagnostics(figsize=(21,12))
plt.show()
print(fit_model.summary())

In [ ]:
#for 00:30
df2=energy.loc[(energy['Hour'] == 0) & (energy['Min'] == 30)]
df2=df2.iloc[:,:2]
sm.tsa.graphics.plot_acf(df2.iloc[:,0], lags=30)
plt.show()

AIC_orders=[]

    # Loop over p values from 0-3
for q in range(4):
      # Loop over q values from 0-6
    for p in range(7):
            # create and fit ARIMA(p,q) model (d=1 because difference of data is stationary (adfuller test)); traning data is
            #given to check the aic
        model = SARIMAX(df2.iloc[0:330,0],exog=df2.iloc[0:330,1],order=(p,1,q),enforce_invertibility=False, enforce_stationarity=False)
        results = model.fit()

            # Append order and results tuple
        AIC_orders.append((p,q,results.aic))

    # DataFrame from AIC_orders
order_df = pd.DataFrame(AIC_orders,
                            columns=['p','q','AIC'])
    #sorting based on the minimum values of AIC
order_df=order_df.sort_values('AIC')
order_df = order_df.reset_index()
del order_df['index']

kn = KneeLocator(range(0,28),order_df['AIC'], curve='convex', direction='increasing') #finding the elbow
h=kn.knee

data = ['00:30',order_df['p'][h], order_df['q'][h]]
df_p_q.loc[len(df_p_q)]=data

    #defining the model based on the optimum p and q that AIC returned.
model= SARIMAX(df2.iloc[0:330,0],exog=df2.iloc[0:330,1], order=(order_df['p'][h],1,order_df['q'][h]), enforce_invertibility=False, enforce_stationarity=False)
results= model.fit()
predictions= results.predict(start =330, end=364,exog= df2.iloc[330:,1])
predictions.index= df2.iloc[330:,0].index
df=df.append(pd.DataFrame(data=predictions))


Error_data=['00:30',rmse(predictions,df2.iloc[330:,0]), rmse(predictions,df2.iloc[330:,0])/average(df2.iloc[330:,0])]
df_error.loc[len(df_error)]=Error_data
print('00:30-static')
results.plot_diagnostics(figsize=(21,12))
plt.show()
print(results.summary())

    #rolling:

train, test = df2.iloc[0:330,0].values, df2.iloc[330:,0].values
train_exo, test_exo= df2.iloc[0:330,1].values, df2.iloc[330:,1].values
history = [x for x in train]
history_exo = [y for y in train_exo]
predictions=list()
for t in range(len(test)):
    model = SARIMAX(history,exog= history_exo,order=(order_df['p'][h],1,order_df['q'][h]),enforce_invertibility=False, enforce_stationarity=False)
    fit_model = model.fit()
    out= fit_model.forecast(exog= test_exo[t])
    yhat = out[0]
    predictions.append(yhat)
    J = test[t]
    s= test_exo[t]
    history.append(J)
    history_exo.append(s)
df_rolling= df_rolling.append(pd.DataFrame(data= predictions, index= df2.iloc[330:,0].index))   #adding new predictions to previous hours values

Error_Rolling_data=['00:30',rmse(predictions,df2.iloc[330,0]), rmse(predictions,df2.iloc[330,0])/average(df2.iloc[330,0])]
df_Rolling_error.loc[len(df_Rolling_error)]=Error_Rolling_data
print('00:30-rolling')
fit_model.plot_diagnostics(figsize=(21,12))
plt.show()
print(fit_model.summary())

In [ ]:
for i in range(1,24):
    for j in range(0,31,30):
        df2=energy.loc[(energy['Hour'] == i) & (energy['Min'] == j) ]
        df2=df2.iloc[:,:2]
        AIC_orders=[]

        # Loop over p values from 0-3
        for q in range(4):
          # Loop over q values from 0-6
            for p in range(7):
                model = SARIMAX(df2.iloc[0:330,0],exog=df2.iloc[0:330,1],order=(p,1,q),enforce_invertibility=False, enforce_stationarity=False)
                results = model.fit()

                # Append order and results tuple
                AIC_orders.append((p,q,results.aic))

        # DataFrame from AIC_orders
        order_df = pd.DataFrame(AIC_orders,
                                columns=['p','q','AIC'])
        #sorting based on the minimum values of AIC
        order_df=order_df.sort_values('AIC')
        order_df = order_df.reset_index()
        del order_df['index']
        kn = KneeLocator(range(0,28),order_df['AIC'], curve='convex', direction='increasing') #finding the elbow
        h=kn.knee

        data = [str(i)+':'+str(j),order_df['p'][h], order_df['q'][h]]
        df_p_q.loc[len(df_p_q)]=data


        #defining the model based on the optimum p and q that AIC returned.
        model= SARIMAX(df2.iloc[0:330,0],exog=df2.iloc[0:330,1], order=(order_df['p'][h],1,order_df['q'][h]), enforce_invertibility=False, enforce_stationarity=False)
        results= model.fit()
        predictions= results.predict(start =330, end=364,exog= df2.iloc[330:,1])
        predictions.index= df2.iloc[330:,0].index
        df=df.append(pd.DataFrame(data=predictions))

        Error_data=[str(i)+':'+str(j),rmse(predictions,df2.iloc[330:,0]), rmse(predictions,df2.iloc[330:,0])/average(df2.iloc[330:,0])]
        df_error.loc[len(df_error)]=Error_data
        print(str(i)+':'+str(j)+'static')
        sm.tsa.graphics.plot_acf(df2.iloc[:,0], lags=30)
        plt.show()
        results.plot_diagnostics(figsize=(21,12))
        plt.show()
        print(results.summary())
        # Rolling forecast
        #split into train and test sets

        train, test = df2.iloc[0:330,0].values, df2.iloc[330:,0].values
        train_exo, test_exo= df2.iloc[0:330,1].values, df2.iloc[330:,1].values
        history = [x for x in train]
        history_exo = [y for y in train_exo]
        predictions=list()
        for t in range(len(test)):
            model = SARIMAX(history,exog= history_exo,order=(order_df['p'][h],1,order_df['q'][h]),enforce_invertibility=False, enforce_stationarity=False)
            fit_model = model.fit()
            out= fit_model.forecast(exog= test_exo[t])
            yhat = out[0]
            predictions.append(yhat)
            J = test[t]
            s= test_exo[t]
            history.append(J)
            history_exo.append(s)
        df_rolling= df_rolling.append(pd.DataFrame(data= predictions, index=df2.iloc[330:,0].index))  #adding new predictions to previous hours values

        Error_Rolling_data=[str(i)+':'+str(j),rmse(predictions,df2.iloc[330:,0]), rmse(predictions,df2.iloc[330:,0])/average(df2.iloc[330:,0])]
        df_Rolling_error.loc[len(df_Rolling_error)]=Error_Rolling_data
        print(str(i)+':'+str(j)+'rolling')
        fit_model.plot_diagnostics(figsize=(21,12))
        plt.show()
        print(fit_model.summary())



In [ ]:
df= df.sort_index()
df_rolling=df_rolling.sort_index()

In [ ]:
fig, ax = plt.subplots(figsize=(21,5))
plt.plot(energy.iloc[15839:,0], color='black',label='Real Energy Consumption- $X_t$')
plt.plot(df.iloc[-1678:-1,0], color= 'red', alpha=0.7,label=' Predicted Energy Consumption static forecast'+ ' $\hat{X_t}$')

plt.plot(df_rolling.iloc[-1678:-1,0], color='green', label='Predicted Energy Consumption- rolling forecast'+ ' $\hat{X_t}$')

plt.title('Prediction for 5 Weeks', fontsize=15)
#plt.ylabel('kwh')
plt.legend()
plt.show()

In [ ]:
mape = mean_absolute_percentage_error(energy.iloc[15839:,0], df.iloc[:,0])*100
print('Mean Absolute Percentage Error for Static forecast:', str(mape)+'%')

In [ ]:
mape = mean_absolute_percentage_error(energy.iloc[15839:,0], df_rolling.iloc[:,0])*100
print('Mean Absolute Percentage Error for Rolling forecast:', str(mape)+'%')

In [ ]:
average(df_error.iloc[:,1])

In [ ]:
average(df_Rolling_error.iloc[:,1])

# Checking for 10 weeks

In [ ]:
#Time: 00:00 (because for this time we have less data for training, it is out of the loop and is done separately)
df2=energy.loc[(energy['Hour'] == 0) & (energy['Min'] == 0)]
df2=df2.iloc[:,:2]
#for 00:00
AIC_orders=[]

# Loop over p values from 0-3
for q in range(4):
  # Loop over q values from 0-6
    for p in range(7):
      	# create and fit ARIMA(p,q) model (d=1 because difference of data is stationary (adfuller test)); traning data is
        #given to check the aic
        model = SARIMAX(df2.iloc[0:294,0],exog=df2.iloc[0:294,1], order=(p,1,q),enforce_invertibility=False, enforce_stationarity=False)
        results = model.fit()

        # Append order and results tuple
        AIC_orders.append((p,q,results.aic))

# DataFrame from AIC_orders
order_df = pd.DataFrame(AIC_orders,
                        columns=['p','q','AIC'])

order_df=order_df.sort_values('AIC')
order_df = order_df.reset_index()
del order_df['index']

kn = KneeLocator(range(0,28),order_df['AIC'], curve='convex', direction='increasing') #finding the elbow
h=kn.knee


data = [{'time':'0','p':order_df['p'][h], 'q':order_df['q'][h]}]

df_p_q = pd.DataFrame(data, columns=['time','p','q'])


#defining the model based on the optimum p and q that AIC returned.
model= SARIMAX(df2.iloc[0:294,0],exog=df2.iloc[0:294,1],order=(order_df['p'][h],1,order_df['q'][h]), enforce_invertibility=False, enforce_stationarity=False)
results= model.fit()

predictions= results.predict(start =294, end=363, exog= df2.iloc[294:,1])
predictions.index= df2.iloc[294:,0].index
df=pd.DataFrame(data=predictions)

Error_data = [{'time':'0','RMSE':rmse(predictions,df2.iloc[294:,0]), 'Normalized RMSE':rmse(predictions,df2.iloc[294:,0])/average(df2.iloc[294:,0])}]
df_error= pd.DataFrame(Error_data, columns=['time','RMSE','Normalized RMSE'])
print('00:00-static')
results.plot_diagnostics(figsize=(21,12))
plt.show()
print(results.summary())
# Rolling forecast

# split into train and test sets
train, test = df2.iloc[0:294,0].values, df2.iloc[294:,0].values
train_exo, test_exo= df2.iloc[0:294,1].values, df2.iloc[294:,1].values
history = [x for x in train]
history_exo = [y for y in train_exo]
predictions=list()
for t in range(len(test)):
    model = SARIMAX(history,exog= history_exo,order=(order_df['p'][h],1,order_df['q'][h]),enforce_invertibility=False, enforce_stationarity=False)
    fit_model = model.fit()
    out= fit_model.forecast(exog= test_exo[t])
    yhat = out[0]
    predictions.append(yhat)
    J = test[t]
    s= test_exo[t]
    history.append(J)
    history_exo.append(s)

df_rolling=pd.DataFrame(data= predictions, index= df2.iloc[294:,0].index)  #adding predicted values to previous predictions

Error_Rolling_data = [{'time':'0','RMSE':rmse(predictions,df2.iloc[294:,0]), 'Normalized RMSE':rmse(predictions,df2.iloc[294:,0])/average(df2.iloc[294:,0])}]
df_Rolling_error= pd.DataFrame(Error_data, columns=['time','RMSE','Normalized RMSE'])
print('00:00-rolling')
fit_model.plot_diagnostics(figsize=(21,12))
plt.show()
print(fit_model.summary())


In [ ]:
#for 00:30
df2=energy.loc[(energy['Hour'] == 0) & (energy['Min'] == 30)]
df2=df2.iloc[:,:2]
sm.tsa.graphics.plot_acf(df2.iloc[:,0], lags=30)
plt.show()

AIC_orders=[]

    # Loop over p values from 0-3
for q in range(4):
      # Loop over q values from 0-6
    for p in range(7):
            # create and fit ARIMA(p,q) model (d=1 because difference of data is stationary (adfuller test)); traning data is
            #given to check the aic
        model = SARIMAX(df2.iloc[0:295,0],exog=df2.iloc[0:295,1],order=(p,1,q),enforce_invertibility=False, enforce_stationarity=False)
        results = model.fit()

            # Append order and results tuple
        AIC_orders.append((p,q,results.aic))

    # DataFrame from AIC_orders
order_df = pd.DataFrame(AIC_orders,
                            columns=['p','q','AIC'])
    #sorting based on the minimum values of AIC
order_df=order_df.sort_values('AIC')
order_df = order_df.reset_index()
del order_df['index']

kn = KneeLocator(range(0,28),order_df['AIC'], curve='convex', direction='increasing') #finding the elbow
h=kn.knee

data = ['00:30',order_df['p'][h], order_df['q'][h]]
df_p_q.loc[len(df_p_q)]=data

    #defining the model based on the optimum p and q that AIC returned.
model= SARIMAX(df2.iloc[0:295,0],exog=df2.iloc[0:295,1], order=(order_df['p'][h],1,order_df['q'][h]), enforce_invertibility=False, enforce_stationarity=False)
results= model.fit()
predictions= results.predict(start =295, end=364,exog= df2.iloc[295:,1])
predictions.index= df2.iloc[295:,0].index
df=df.append(pd.DataFrame(data=predictions))


Error_data=['00:30',rmse(predictions,df2.iloc[295:,0]), rmse(predictions,df2.iloc[295:,0])/average(df2.iloc[295:,0])]
df_error.loc[len(df_error)]=Error_data
print('00:30-static')
results.plot_diagnostics(figsize=(21,12))
plt.show()
print(results.summary())

    #rolling:
train, test = df2.iloc[0:295,0].values, df2.iloc[295:,0].values
train_exo, test_exo= df2.iloc[0:295,1].values, df2.iloc[295:,1].values
history = [x for x in train]
history_exo = [y for y in train_exo]
predictions=list()
for t in range(len(test)):
    model = SARIMAX(history,exog= history_exo,order=(order_df['p'][h],1,order_df['q'][h]),enforce_invertibility=False, enforce_stationarity=False)
    fit_model = model.fit()
    out= fit_model.forecast(exog= test_exo[t])
    yhat = out[0]
    predictions.append(yhat)
    J = test[t]
    s= test_exo[t]
    history.append(J)
    history_exo.append(s)
df_rolling= df_rolling.append(pd.DataFrame(data= predictions, index= df2.iloc[295:,0].index))   #adding new predictions to previous hours values

Error_Rolling_data=['00:30',rmse(predictions,df2.iloc[295,0]), rmse(predictions,df2.iloc[295,0])/average(df2.iloc[295,0])]
df_Rolling_error.loc[len(df_Rolling_error)]=Error_Rolling_data
print('00:30-rolling')
fit_model.plot_diagnostics(figsize=(21,12))
plt.show()
print(fit_model.summary())

In [ ]:
for i in range(1,24):
    for j in range(0,31,30):
        df2=energy.loc[(energy['Hour'] == i) & (energy['Min'] == j) ]
        df2=df2.iloc[:,:2]
        AIC_orders=[]

        # Loop over p values from 0-3
        for q in range(4):
          # Loop over q values from 0-6
            for p in range(7):
                model = SARIMAX(df2.iloc[0:295,0],exog=df2.iloc[0:295,1],order=(p,1,q),enforce_invertibility=False, enforce_stationarity=False)
                results = model.fit()

                # Append order and results tuple
                AIC_orders.append((p,q,results.aic))

        # DataFrame from AIC_orders
        order_df = pd.DataFrame(AIC_orders,
                                columns=['p','q','AIC'])
        #sorting based on the minimum values of AIC
        order_df=order_df.sort_values('AIC')
        order_df = order_df.reset_index()
        del order_df['index']
        kn = KneeLocator(range(0,28),order_df['AIC'], curve='convex', direction='increasing') #finding the elbow
        h=kn.knee

        data = [str(i)+':'+str(j),order_df['p'][h], order_df['q'][h]]
        df_p_q.loc[len(df_p_q)]=data


        #defining the model based on the optimum p and q that AIC returned.
        model= SARIMAX(df2.iloc[0:295,0],exog=df2.iloc[0:295,1], order=(order_df['p'][h],1,order_df['q'][h]), enforce_invertibility=False, enforce_stationarity=False)
        results= model.fit()
        predictions= results.predict(start =295, end=364,exog= df2.iloc[295:,1])
        predictions.index= df2.iloc[295:,0].index
        df=df.append(pd.DataFrame(data=predictions))

        Error_data=[str(i)+':'+str(j),rmse(predictions,df2.iloc[295:,0]), rmse(predictions,df2.iloc[295:,0])/average(df2.iloc[295:,0])]
        df_error.loc[len(df_error)]=Error_data
        print(str(i)+':'+str(j)+'static')
        sm.tsa.graphics.plot_acf(df2.iloc[:,0], lags=30)
        plt.show()
        results.plot_diagnostics(figsize=(21,12))
        plt.show()
        print(results.summary())
        # Rolling forecast
        #split into train and test sets

        train, test = df2.iloc[0:295,0].values, df2.iloc[295:,0].values
        train_exo, test_exo= df2.iloc[0:295,1].values, df2.iloc[295:,1].values
        history = [x for x in train]
        history_exo = [y for y in train_exo]
        predictions=list()
        for t in range(len(test)):
            model = SARIMAX(history,exog= history_exo,order=(order_df['p'][h],1,order_df['q'][h]),enforce_invertibility=False, enforce_stationarity=False)
            fit_model = model.fit()
            out= fit_model.forecast(exog= test_exo[t])
            yhat = out[0]
            predictions.append(yhat)
            J = test[t]
            s= test_exo[t]
            history.append(J)
            history_exo.append(s)
        df_rolling= df_rolling.append(pd.DataFrame(data= predictions, index=df2.iloc[295:,0].index))  #adding new predictions to previous hours values

        Error_Rolling_data=[str(i)+':'+str(j),rmse(predictions,df2.iloc[295:,0]), rmse(predictions,df2.iloc[295:,0])/average(df2.iloc[295:,0])]
        df_Rolling_error.loc[len(df_Rolling_error)]=Error_Rolling_data
        print(str(i)+':'+str(j)+'rolling')
        fit_model.plot_diagnostics(figsize=(21,12))
        plt.show()
        print(fit_model.summary())

In [ ]:
df= df.sort_index()
df_rolling=df_rolling.sort_index()

In [ ]:
fig, ax = plt.subplots(figsize=(21,5))
plt.plot(energy.iloc[14159:,0], color='black',label='Real Energy Consumption- $X_t$')
plt.plot(df.iloc[:,0], color= 'red', alpha=0.7,label=' Predicted Energy Consumption- static forecast'+ ' $\hat{X_t}$')
plt.plot(df_rolling.iloc[-3360:-1,0], color='green', label='Predicted Energy Consumption- rolling forecast'+ ' $\hat{X_t}$')
plt.title('Prediction for 10 Weeks', fontsize=15)
#plt.ylabel('kwh')
plt.xlabel('Date', fontsize=15)
plt.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(21,5))
plt.plot(energy.iloc[15839:,0], color='black',label='Real Energy Consumption- $X_t$')
plt.plot(df.iloc[-1678:-1,0], color= 'red', alpha=0.7,label=' Predicted Energy Consumption static forecast'+ ' $\hat{X_t}$')

plt.plot(df_rolling.iloc[-1678:-1,0], color='green', label='Predicted Energy Consumption- rolling forecast'+ ' $\hat{X_t}$')

plt.title('Prediction for 5 Weeks', fontsize=15)
#plt.ylabel('kwh')
plt.legend()
plt.show()

In [ ]:
mape = mean_absolute_percentage_error(energy.iloc[14159:,0], df.iloc[:,0])*100
print('Mean Absolute Percentage Error for Static forecast:', str(mape)+'%')

In [ ]:
mape = mean_absolute_percentage_error(energy.iloc[14159:,0], df_rolling.iloc[:,0])*100
print('Mean Absolute Percentage Error for Rolling forecast:', str(mape)+'%')

In [ ]:
average(df_error.iloc[:,1])

In [ ]:
average(df_Rolling_error.iloc[:,1])

# Whitout Exogenous factor (Temperature)

In [ ]:
#Time: 00:00 (because for this time we have less data for training, it is out of the loop and is done separately)
df2=energy.loc[(energy['Hour'] == 0) & (energy['Min'] == 0)]
df2=df2.iloc[:,:2]
df2

In [ ]:
#for 00:00
AIC_orders=[]

# Loop over p values from 0-3
for q in range(4):
  # Loop over q values from 0-6
    for p in range(7):
      	# create and fit ARIMA(p,q) model (d=1 because difference of data is stationary (adfuller test)); traning data is
        #given to check the aic
        model = SARIMAX(df2.iloc[0:357,0], order=(p,1,q),enforce_invertibility=False, enforce_stationarity=False)
        results = model.fit()

        # Append order and results tuple
        AIC_orders.append((p,q,results.aic))

# DataFrame from AIC_orders
order_df = pd.DataFrame(AIC_orders,
                        columns=['p','q','AIC'])

order_df=order_df.sort_values('AIC')
order_df = order_df.reset_index()
del order_df['index']

kn = KneeLocator(range(0,28),order_df['AIC'], curve='convex', direction='increasing') #finding the elbow
h=kn.knee


data = [{'time':'0','p':order_df['p'][h], 'q':order_df['q'][h]}]

df_p_q = pd.DataFrame(data, columns=['time','p','q'])

#defining the model based on the optimum p and q that AIC returned.
model= SARIMAX(df2.iloc[0:357,0],order=(order_df['p'][h],1,order_df['q'][h]), enforce_invertibility=False, enforce_stationarity=False)
results= model.fit()

predictions= results.predict(start =357, end=363)
predictions.index= df2.iloc[357:,0].index
df=pd.DataFrame(data=predictions)

Error_data = [{'time':'0','RMSE':rmse (predictions,df2.iloc[357:,0]), 'Normalized RMSE':rmse(predictions,df2.iloc[357:,0])/average(df2.iloc[357:,0])}]
df_error= pd.DataFrame(Error_data, columns=['time','RMSE','Normalized RMSE'])
print('00:00-static')
results.plot_diagnostics(figsize=(21,12))
plt.show()
print(results.summary())
# Rolling forecast

train, test = df2.iloc[0:357,0].values, df2.iloc[357:,0].values

history = [x for x in train]

predictions=list()
for t in range(len(test)):
    model = SARIMAX(history,order=(order_df['p'][h],1,order_df['q'][h]),enforce_invertibility=False, enforce_stationarity=False)
    fit_model = model.fit()
    out= fit_model.forecast()
    yhat = out[0]
    predictions.append(yhat)
    J = test[t]
    history.append(J)

df_rolling=pd.DataFrame(data= predictions, index= df2.iloc[357:,0].index)  #adding predicted values to previous predictions

Error_Rolling_data = [{'time':'0','RMSE':rmse(predictions,df2.iloc[357:,0]), 'Normalized RMSE':rmse(predictions,df2.iloc[357:,0])/average(df2.iloc[357:,0])}]
df_Rolling_error= pd.DataFrame(Error_data, columns=['time','RMSE','Normalized RMSE'])
print('00:00-rolling')
fit_model.plot_diagnostics(figsize=(21,12))
plt.show()
print(fit_model.summary())


In [ ]:
#for 00:30
df2=energy.loc[(energy['Hour'] == 0) & (energy['Min'] == 30)]
df2=df2.iloc[:,:2]
sm.tsa.graphics.plot_acf(df2.iloc[:,0], lags=30)
plt.show()

AIC_orders=[]

    # Loop over p values from 0-3
for q in range(4):
      # Loop over q values from 0-6
    for p in range(7):
            # create and fit ARIMA(p,q) model (d=1 because difference of data is stationary (adfuller test)); traning data is
            #given to check the aic
        model = SARIMAX(df2.iloc[0:358,0],order=(p,1,q),enforce_invertibility=False, enforce_stationarity=False)
        results = model.fit()

            # Append order and results tuple
        AIC_orders.append((p,q,results.aic))

    # DataFrame from AIC_orders
order_df = pd.DataFrame(AIC_orders,
                            columns=['p','q','AIC'])
    #sorting based on the minimum values of AIC
order_df=order_df.sort_values('AIC')
order_df = order_df.reset_index()
del order_df['index']

kn = KneeLocator(range(0,28),order_df['AIC'], curve='convex', direction='increasing') #finding the elbow
h=kn.knee

data = ['00:30',order_df['p'][h], order_df['q'][h]]
df_p_q.loc[len(df_p_q)]=data


    #defining a list to collect predicted values.
    #defining the model based on the optimum p and q that AIC returned.
model= SARIMAX(df2.iloc[0:358,0], order=(order_df['p'][h],1,order_df['q'][h]), enforce_invertibility=False, enforce_stationarity=False)
results= model.fit()
predictions= results.predict(start =358, end=364)
predictions.index= df2.iloc[358:,0].index
df=df.append(pd.DataFrame(data=predictions))


Error_data=['00:30',rmse(predictions,df2.iloc[358:,0]), rmse(predictions,df2.iloc[358:,0])/average(df2.iloc[358:,0])]
df_error.loc[len(df_error)]=Error_data
print('00:30-static')
results.plot_diagnostics(figsize=(21,12))
plt.show()
print(results.summary())

    #rolling:

train, test = df2.iloc[0:358,0].values, df2.iloc[358:,0].values
history = [x for x in train]
predictions=list()
for t in range(len(test)):
    model = SARIMAX(history,order=(order_df['p'][h],1,order_df['q'][h]),enforce_invertibility=False, enforce_stationarity=False)
    fit_model = model.fit()
    out= fit_model.forecast()
    yhat = out[0]
    predictions.append(yhat)
    J = test[t]
    history.append(J)
df_rolling= df_rolling.append(pd.DataFrame(data= predictions, index= df2.iloc[358:,0].index))   #adding new predictions to previous hours values

Error_Rolling_data=['00:30',rmse(predictions,df2.iloc[358,0]), rmse(predictions,df2.iloc[358,0])/average(df2.iloc[358,0])]
df_Rolling_error.loc[len(df_Rolling_error)]=Error_Rolling_data
print('00:30-rolling')
fit_model.plot_diagnostics(figsize=(21,12))
plt.show()
print(fit_model.summary())

In [ ]:
for i in range(1,24):
    for j in range(0,31,30):
        df2=energy.loc[(energy['Hour'] == i) & (energy['Min'] == j) ]
        df2=df2.iloc[:,:2]
        AIC_orders=[]

        # Loop over p values from 0-3
        for q in range(4):
          # Loop over q values from 0-6
            for p in range(7):
                model = SARIMAX(df2.iloc[0:358,0],order=(p,1,q),enforce_invertibility=False, enforce_stationarity=False)
                results = model.fit()

                # Append order and results tuple
                AIC_orders.append((p,q,results.aic))

        # DataFrame from AIC_orders
        order_df = pd.DataFrame(AIC_orders,
                                columns=['p','q','AIC'])
        #sorting based on the minimum values of AIC
        order_df=order_df.sort_values('AIC')
        order_df = order_df.reset_index()
        del order_df['index']
        kn = KneeLocator(range(0,28),order_df['AIC'], curve='convex', direction='increasing') #finding the elbow
        h=kn.knee

        data = [str(i)+':'+str(j),order_df['p'][h], order_df['q'][h]]
        df_p_q.loc[len(df_p_q)]=data


        #defining the model based on the optimum p and q that AIC returned.
        model= SARIMAX(df2.iloc[0:358,0], order=(order_df['p'][h],1,order_df['q'][h]), enforce_invertibility=False, enforce_stationarity=False)
        results= model.fit()
        predictions= results.predict(start =358, end=364)
        predictions.index= df2.iloc[358:,0].index
        df=df.append(pd.DataFrame(data=predictions))

        Error_data=[str(i)+':'+str(j),rmse(predictions,df2.iloc[358:,0]), rmse(predictions,df2.iloc[358:,0])/average(df2.iloc[358:,0])]
        df_error.loc[len(df_error)]=Error_data
        print(str(i)+':'+str(j)+'static')
        sm.tsa.graphics.plot_acf(df2.iloc[:,0], lags=30)
        plt.show()
        results.plot_diagnostics(figsize=(21,12))
        plt.show()
        print(results.summary())
        # Rolling forecast
        #split into train and test sets

        train, test = df2.iloc[0:358,0].values, df2.iloc[358:,0].values
        history = [x for x in train]
        predictions=list()
        for t in range(len(test)):
            model = SARIMAX(history,order=(order_df['p'][h],1,order_df['q'][h]),enforce_invertibility=False, enforce_stationarity=False)
            fit_model = model.fit()
            out= fit_model.forecast()
            yhat = out[0]
            predictions.append(yhat)
            J = test[t]
            history.append(J)
        df_rolling= df_rolling.append(pd.DataFrame(data= predictions, index=df2.iloc[358:,0].index))  #adding new predictions to previous hours values

        Error_Rolling_data=[str(i)+':'+str(j),rmse(predictions,df2.iloc[358:,0]), rmse(predictions,df2.iloc[358:,0])/average(df2.iloc[358:,0])]
        df_Rolling_error.loc[len(df_Rolling_error)]=Error_Rolling_data
        print(str(i)+':'+str(j)+'rolling')
        fit_model.plot_diagnostics(figsize=(21,12))
        plt.show()
        print(fit_model.summary())


In [ ]:
df= df.sort_index()

In [ ]:
df_rolling=df_rolling.sort_index()
fig, ax = plt.subplots(figsize=(21,5))
plt.plot(energy.iloc[17183:,0], color='black',label='Real Energy Consumption, $X_t$')
plt.plot(df.iloc[:,0], color= 'red', alpha=0.7,label=' Predicted Energy Consumption- Static Forecast'+ ' $\hat{X_t}$')
plt.plot(df_rolling.iloc[:,0], color='lightgreen', label='Predicted Energy Consumption- Rolling forecast'+ ' $\hat{X_t}$')

#plt.plot(energy.iloc[17184:,1], color='orange', label='Temperatue')
#plt.title('Prediction for 1 Week (Model Without Exogenous Factor)', fontsize=15)
#plt.ylabel('Energy (KWh)')
plt.xlabel('Date', fontsize=15)
plt.legend()
plt.show()

In [ ]:
#calculating mean absolute percentage error in order to compare with other ML models
mape = mean_absolute_percentage_error(energy.iloc[17183:,0], df_rolling.iloc[:,0])*100
print('Mean Absolute Percentage Error for Rolling forecast:', str(mape)+'%')

In [ ]:
mape = mean_absolute_percentage_error(energy.iloc[17183:,0], df.iloc[:,0])*100
print('Mean Absolute Percentage Error for Static forecast:', str(mape)+'%')

In [ ]:
rmse = rmse(energy.iloc[17183:,0], df_rolling.iloc[:,0])
print('RMSE for Rolling forecast:', str(rmse)+'%')

In [ ]:
average(df_error.iloc[:,1])

In [ ]:
average(df_Rolling_error.iloc[:,1])